In [ ]:
import sys
sys.path.append("/Users/user/question-retrieval-KIPerWeb/")
from utils import *
import pandas as pd
import re
from sentence_transformers import SentenceTransformer
import json
from nltk.stem.cistem import Cistem
from nltk.corpus import stopwords
from ranx import Qrels, evaluate, Run, compare
import swifter
import json
import requests
from sklearn.preprocessing import LabelEncoder
from trectools import TrecPoolMaker, TrecRun
from tqdm import tqdm
from collections import Counter
import spacy
from io import StringIO
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

## Read the QB's sample

In [ ]:
# Read preprocessed data
qb = pd.read_csv("/Users/user/question-retrieval-KIPerWeb/testbeds/samples/qb_stratified-sample.csv").fillna("N/A")
qb = qb.drop(columns=['docid'])
qb.rename(columns={"category":"topic_label_de_fixed"}, inplace=True)
qb['docid'] = qb.index
qb['content'] = qb['content'].apply(lambda x:remove_stop_words(x, 'german'))


# Preprocessing
# Models for preprocessing
nlp = spacy.load("de_core_news_sm")
tagger_de = ht.HanoverTagger('morphmodel_ger.pgz')

qb['content'] = qb['content'].apply(lambda x : preprocess_documents(x, tagger_de))
qb

In [ ]:
print(qb['text'].iloc[3], qb['answers_content'].iloc[3])
print(qb['content'].iloc[3])

## Calculate Embeddings and Initialize NMSLIB Indices

In [ ]:
model_1 = SentenceTransformer("sentence-transformers/paraphrase-multilingual-mpnet-base-v2")
model_2 = SentenceTransformer("aari1995/German_Semantic_STS_V2")
# model_2.tokenizer.pad_token = model_2.tokenizer.eos_token

model_3 = SentenceTransformer("sentence-transformers/LaBSE")
model_4 = SentenceTransformer("PM-AI/bi-encoder_msmarco_bert-base_german")
model_5 = SentenceTransformer("efederici/e5-base-multilingual-4096")
model_6 = SentenceTransformer("intfloat/multilingual-e5-base")
model_7 = SentenceTransformer("clips/mfaq")
model_8 = SentenceTransformer("PM-AI/sts_paraphrase_xlm-roberta-base_de-en")
model_9 = SentenceTransformer("deutsche-telekom/gbert-large-paraphrase-euclidean")
model_10 = SentenceTransformer("LLukas22/all-MiniLM-L12-v2-embedding-all")
model_11 = SentenceTransformer("LLukas22/paraphrase-multilingual-mpnet-base-v2-embedding-all")
model_12 = SentenceTransformer("sentence-transformers/distiluse-base-multilingual-cased-v1")
model_13 = SentenceTransformer("sentence-transformers/distiluse-base-multilingual-cased-v2")
model_14 = SentenceTransformer("deutsche-telekom/gbert-large-paraphrase-cosine")
model_15 = SentenceTransformer("shibing624/text2vec-base-multilingual")
model_16 = SentenceTransformer("Sahajtomar/German-semantic")
model_17 = SentenceTransformer("setu4993/LaBSE")
model_18 = SentenceTransformer("symanto/sn-xlm-roberta-base-snli-mnli-anli-xnli")
model_19 = SentenceTransformer("and-effect/musterdatenkatalog_clf")
model_20 = SentenceTransformer("nblokker/debatenet-2-cat")
model_21 = SentenceTransformer("setu4993/LEALLA-large")
model_22 = SentenceTransformer("dell-research-harvard/lt-wikidata-comp-de")
model_23 = SentenceTransformer("ef-zulla/e5-multi-sml-torch")
model_24 = SentenceTransformer("barisaydin/text2vec-base-multilingual")
model_25 = SentenceTransformer("meta-llama/Llama-2-7b-chat-hf")
model_25.tokenizer.pad_token = model_25.tokenizer.eos_token


In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import nmslib
from sentence_transformers import SentenceTransformer

def create_index(documents_set, model, **kwargs):
    '''
    Create index with TF-IDF weighted average embeddings.

    documents_set: A dataframe with all the context to be indexed.
    model: A language model to calculate embeddings.
    index_path: The path for storing the index in case one needs to store it.
    '''
    # Initialize TF-IDF Vectorizer and compute TF-IDF matrix
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(documents_set['content'].tolist())

    # Initialize the nmslib index
    index = nmslib.init(method='hnsw', space='cosinesimil')

    # Function to compute TF-IDF weighted average embeddings
    def tfidf_weighted_embeddings(text, vectorizer, tfidf_row):
        # Tokenize text and get embeddings
        words = text.split()
        word_embeddings = model.encode(words, convert_to_tensor=True)

        # Convert embeddings to numpy arrays if they are tensors
        if hasattr(word_embeddings, 'cpu'):
            word_embeddings = word_embeddings.cpu().numpy()

        # Compute weights
        weights = np.array([tfidf_row[0, vectorizer.vocabulary_.get(word.lower(), 0)] for word in words])

        # Calculate weighted average
        weighted_embeddings = word_embeddings * weights[:, np.newaxis]
        weighted_average = np.sum(weighted_embeddings, axis=0) / np.sum(weights)
        return weighted_average

    # Compute TF-IDF weighted embeddings for each document
    document_set_embeddings = []
    for i, text in enumerate(documents_set['content'].tolist()):
        weighted_avg_embeddings = tfidf_weighted_embeddings(text, tfidf_vectorizer, tfidf_matrix[i])
        document_set_embeddings.append(weighted_avg_embeddings)

    # Add data points to the index
    index.addDataPointBatch(document_set_embeddings)
    index.createIndex({'post': 2})

    # Save index if path is provided
    if 'index_path' in kwargs:
        index.saveIndex(kwargs['index_path'])

    return index



index_1 = create_index(qb, model_1, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices-tfidf/sentence-transformers-paraphrase-multilingual-mpnet-base-v2.nmslib")
index_2 = create_index(qb, model_2, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices-tfidf/aari1995-German_Semantic_STS_V2.nmslib")
index_3 = create_index(qb, model_3, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices-tfidf/sentence-transformers-LaBSE.nmslib")
index_4 = create_index(qb, model_4, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices-tfidf/PM-AI-bi-encoder_msmarco_bert-base_german.nmslib")
index_5 = create_index(qb, model_5, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices-tfidf/efederici-e5-base-multilingual-4096.nmslib")
index_6 = create_index(qb, model_6, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices-tfidf/intfloat-multilingual-e5-base.nmslib")
index_7 = create_index(qb, model_7, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices-tfidf/clips-mfaq.nmslib")
index_8 = create_index(qb, model_8, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices-tfidf/PM-AI-sts_paraphrase_xlm-roberta-base_de-en.nmslib")
index_9 = create_index(qb, model_9, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices-tfidf/deutsche-telekom-gbert-large-paraphrase-euclidean.nmslib")
index_10 = create_index(qb, model_10, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices-tfidf/LLukas22-all-MiniLM-L12-v2-embedding-all.nmslib")
index_11 = create_index(qb, model_11, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices-tfidf/LLukas22-paraphrase-multilingual-mpnet-base-v2-embedding-all.nmslib")
index_12 = create_index(qb, model_12, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices-tfidf/sentence-transformers-distiluse-base-multilingual-cased-v1.nmslib")
index_13 = create_index(qb, model_13, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices-tfidf/sentence-transformers-distiluse-base-multilingual-cased-v2.nmslib")
index_14 = create_index(qb, model_14, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices-tfidf/deutsche-telekom-gbert-large-paraphrase-cosine.nmslib")
index_15 = create_index(qb, model_15, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices-tfidf/shibing624-text2vec-base-multilingual.nmslib")
index_16 = create_index(qb, model_16, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices-tfidf/Sahajtomar-German-semantic.nmslib")
index_17 = create_index(qb, model_17, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices-tfidf/setu4993-LaBSE.nmslib")
index_18 = create_index(qb, model_18, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices-tfidf/symanto-sn-xlm-roberta-base-snli-mnli-anli-xnli.nmslib")
index_19 = create_index(qb, model_19, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices-tfidf/and-effect-musterdatenkatalog_clf.nmslib")
index_20 = create_index(qb, model_20, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices-tfidf/nblokker-debatenet-2-cat.nmslib")
index_21 = create_index(qb, model_21, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices-tfidf/setu4993-LEALLA-large.nmslib")
index_22 = create_index(qb, model_22, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices-tfidf/dell-research-harvard-lt-wikidata-comp-de.nmslib")
index_23 = create_index(qb, model_23, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices-tfidf/ef-zulla-e5-multi-sml-torch.nmslib")
index_24 = create_index(qb, model_24, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices-tfidf/barisaydin-text2vec-base-multilingual.nmslib")
index_25 = create_index(qb, model_25, index_path="/Users/user/question-retrieval-KIPerWeb/nmslib_indices-tfidf/meta-llama-Llama-2-7b-chat-hf.nmslib")


# # Or load them:

# index_1 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices-tfidf/sentence-transformers-paraphrase-multilingual-mpnet-base-v2.nmslib")
# index_2 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices-tfidf/aari1995-German_Semantic_STS_V2.nmslib")
# index_3 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices-tfidf/sentence-transformers-LaBSE.nmslib")
# index_4 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices-tfidf/PM-AI-bi-encoder_msmarco_bert-base_german.nmslib")
# index_5 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices-tfidf/efederici-e5-base-multilingual-4096.nmslib")
# index_6 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices-tfidf/intfloat-multilingual-e5-base.nmslib")
# index_7 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices-tfidf/clips-mfaq.nmslib")
# index_8 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices-tfidf/PM-AI-sts_paraphrase_xlm-roberta-base_de-en.nmslib")
# index_9 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices-tfidf/deutsche-telekom-gbert-large-paraphrase-euclidean.nmslib")
# index_10 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices-tfidf/LLukas22-all-MiniLM-L12-v2-embedding-all.nmslib")
# index_11 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices-tfidf/LLukas22-paraphrase-multilingual-mpnet-base-v2-embedding-all.nmslib")
# index_12 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices-tfidf/sentence-transformers-distiluse-base-multilingual-cased-v1.nmslib")
# index_13 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices-tfidf/sentence-transformers-distiluse-base-multilingual-cased-v2.nmslib")
# index_14 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices-tfidf/deutsche-telekom-gbert-large-paraphrase-cosine.nmslib")
# index_15 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices-tfidf/shibing624-text2vec-base-multilingual.nmslib")
# index_16 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices-tfidf/Sahajtomar-German-semantic.nmslib")
# index_17 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices-tfidf/setu4993-LaBSE.nmslib")
# index_18 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices-tfidf/symanto-sn-xlm-roberta-base-snli-mnli-anli-xnli.nmslib")
# index_19 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices-tfidf/and-effect-musterdatenkatalog_clf.nmslib")
# index_20 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices-tfidf/nblokker-debatenet-2-cat.nmslib")
# index_21 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices-tfidf/setu4993-LEALLA-large.nmslib")
# index_22 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices-tfidf/dell-research-harvard-lt-wikidata-comp-de.nmslib")
# index_23 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices-tfidf/ef-zulla-e5-multi-sml-torch.nmslib")
# index_24 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices-tfidf/barisaydin-text2vec-base-multilingual.nmslib")
# index_25 = load_nmslib_index("/Users/user/question-retrieval-KIPerWeb/nmslib_indices-tfidf/meta-llama-Llama-2-7b-chat-hf.nmslib")


## Run Queries Against Search Indices

In [ ]:


def search(query, index, model, df, **kwargs):
    """
    Approximate nearest neighbor search
    
    Parameters
    ----------
    query: A string query
    index: An NMSLIB index
    model: A language model
    df: The content against the query will be runned
    run_path: the path where the results will be stored (in JSON format)
    k: The number of elements to be indexed by the search, by default is 10.
    
    Returns
    -------
    """
    # Calculate embeddings with the language model
    # try:
    #     query_embeddings = model.encode(query)
    # except ValueError:
    #     # model.tokenizer.eos_token = model.tokenizer.eos_token
    #     model.tokenizer.eos_token = "[EOS]"
    #     model.tokenizer.mask_token = '[MASK]'
    #     model.tokenizer.bos_token = '[BOS]'
    #     # model.tokenizer.pad_token = model.tokenizer.pad_token_id
    #     model.tokenizer.pad_token = '[PAD]'
    #     model.tokenizer.cls_token = "[CLS]"
    #     model.tokenizer.sep_token = "[SEP]"
    # Initialize TF-IDF Vectorizer
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_vectorizer.fit(df['content'].tolist())

    # Initialize TF-IDF Vectorizer
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_vectorizer.fit(df['content'].tolist())

    # Function to compute TF-IDF weighted average embeddings
    def tfidf_weighted_embeddings(text, vectorizer):
        # Tokenize text and get embeddings
        words = text.split()
        word_embeddings = model.encode(words, convert_to_tensor=True)

        # Compute weights
        weights = torch.tensor([vectorizer.idf_[vectorizer.vocabulary_.get(word.lower(), 0)] for word in words])

        # Perform the operations using PyTorch
        if word_embeddings.dim() == 1:  # In case the result is a 1D tensor
            word_embeddings = word_embeddings.unsqueeze(0)
        if weights.dim() == 1:  # Also ensure weights are 2D
            weights = weights.unsqueeze(0)
        weighted_embeddings = word_embeddings * weights.T
        weighted_average = torch.sum(weighted_embeddings, dim=0) / torch.sum(weights)
        return weighted_average.cpu().numpy() if weighted_average.is_cuda else weighted_average.numpy()



    # query_embeddings = model.encode(query)
    # Calculate TF-IDF weighted average embeddings for the query
    query_embeddings = tfidf_weighted_embeddings(query, tfidf_vectorizer)


    model_name = Path(str(model.tokenizer).split()[0].split("=")[1].replace("'","").replace(",","")).parts[-1:][0]
    # With the index, make a query and approximate its 25 nearest neighbors
    if 'k' in kwargs:
        k = kwargs['k']
    else:
        k = df.shape[0]
    ids, distances = index.knnQuery(query_embeddings, k=k)
    # Process the output
    indices_and_weights = list(zip(ids, distances))
    
    nmslib_indices = [e[0] for e in indices_and_weights]
    results_df = df.iloc[nmslib_indices]
    retrieved_question_ids = [e for e in results_df['docid'].tolist()]
    # print(retrieved_question_ids)
    # TODO: check if it is necessary to reset the indices in the DF,
    # in the original DF there were repeated elements in the content column
    # for consistency with the original data, the new reference id will be removed
    results = df.query("docid in @retrieved_question_ids")[:k]#.reset_index(drop=True)[:k]
    # results['query_embedding'] = [query_embeddings]*results.shape[0] # this is for plotting, still needs to be reduced
    results = results[['docid', 'onlinetest_title', 'question_type_id', 'question_type_name', 
                       'answer_type_id', 'answer_type_name', 'text', 
                       'correct_answers_temp','variable', 'points', 'answers',
                       'source', 'level_difficulty', 
                       'topic_label_de_fixed', 'related_topics']]



    results['answers'] = results['answers'].apply(to_dic)
    results['related_topics'] = results['related_topics'].apply(to_dic)

    
    results['points'] = results['points'].fillna('N/A')
    results['onlinetest_title'] = results['onlinetest_title'].fillna('N/A')
    results['question_type_id'] = results['question_type_id'].fillna('N/A')
    results['question_type_name'] = results['question_type_name'].fillna('N/A')
    results['variable'] = results['variable'].fillna('N/A')
    results['correct_answers_temp'] = results['correct_answers_temp'].fillna('N/A')

    
    topic_filters = list(set(results['topic_label_de_fixed'].tolist()))
    answer_type_filters = list(set(results['answer_type_name'].tolist()))
    question_type_filters = list(set(results['question_type_name'].tolist()))
    question_difficulty_filters = list(set(results['level_difficulty'].tolist()))
    source_filters = list(set(results['source'].tolist()))
    points_filters = list(set(results['points'].tolist()))

    results = [e[1] for e in results.T.to_dict().items()]
    
    for i, j in zip(results, indices_and_weights):
        i['cosine_distance'] = str(j[1])
    for e in results:
        e['query']=str(query)

    filter_data = {
        'topic_filters':topic_filters, 
        'answer_type_filters': answer_type_filters,
        'question_type_filters': question_type_filters,
        'question_difficulty_filters': question_difficulty_filters,
        'source_filters':source_filters, 
        'points_filters':points_filters
        }
    

    results = {
        'filter_data' : filter_data,
        'search_output' : results
        }

    if 'run_path' in kwargs:
        save_to_json([dict(e) for e in results['search_output']],
                     kwargs['run_path'], model_name=model_name) # corregir este parseo ya no aplica!!!
    else:
        pass
    return results




query_test = search('Gesundheitswesen'.lower(), index_1, model_1, qb, k=200)['search_output']
query_test

In [ ]:
## We could use queries:
# queries_file = pd.read_csv("/Users/user/question-retrieval-KIPerWeb/testbeds/topics_experiments/topics/topics.csv")
queries_file = pd.read_csv("/Users/user/question-retrieval-KIPerWeb/testbeds/queries_experiments/queries/queries.csv")
queries_file['queries'] = queries_file['queries'].apply(clean_text)



print("-> Number of queries:",len(queries_file['queries'].tolist()))

queries = queries_file['queries'].tolist()
query_ids = queries_file['qid'].tolist()
queries_file

## Generate searches and store them as JSON files

In [ ]:
path = '/Users/user/question-retrieval-KIPerWeb/runs/ANN_LM-TFIDF-Weighted-AVG/json_format/'
progress_bar = tqdm(zip(queries, query_ids), total=len(queries), desc="Processing")

for i, j in progress_bar:
    progress_bar.set_description(f"Processing query: {i}")
    search(i, index_1, model_1, qb, k=100, run_path=f'{path}query_{j}/run_01.json')
    search(i, index_2, model_2, qb, k=100, run_path=f'{path}query_{j}/run_02.json')
    search(i, index_3, model_3, qb, k=100, run_path=f'{path}query_{j}/run_03.json')
    search(i, index_4, model_4, qb, k=100, run_path=f'{path}query_{j}/run_04.json')
    search(i, index_5, model_5, qb, k=100, run_path=f'{path}query_{j}/run_05.json')
    search(i, index_6, model_6, qb, k=100, run_path=f'{path}query_{j}/run_06.json')
    search(i, index_7, model_7, qb, k=100, run_path=f'{path}query_{j}/run_07.json')
    search(i, index_8, model_8, qb, k=100, run_path=f'{path}query_{j}/run_08.json')
    search(i, index_9, model_9, qb, k=100, run_path=f'{path}query_{j}/run_09.json')
    search(i, index_10, model_10, qb, k=100, run_path=f'{path}query_{j}/run_10.json')
    search(i, index_11, model_11, qb, k=100, run_path=f'{path}query_{j}/run_11.json')
    search(i, index_12, model_12, qb, k=100, run_path=f'{path}query_{j}/run_12.json')
    search(i, index_13, model_13, qb, k=100, run_path=f'{path}query_{j}/run_13.json')
    search(i, index_14, model_14, qb, k=100, run_path=f'{path}query_{j}/run_14.json')
    search(i, index_15, model_15, qb, k=100, run_path=f'{path}query_{j}/run_15.json')
    search(i, index_16, model_16, qb, k=100, run_path=f'{path}query_{j}/run_16.json')
    search(i, index_17, model_17, qb, k=100, run_path=f'{path}query_{j}/run_17.json')
    search(i, index_18, model_18, qb, k=100, run_path=f'{path}query_{j}/run_18.json')
    search(i, index_19, model_19, qb, k=100, run_path=f'{path}query_{j}/run_19.json')
    search(i, index_20, model_20, qb, k=100, run_path=f'{path}query_{j}/run_20.json')
    search(i, index_21, model_21, qb, k=100, run_path=f'{path}query_{j}/run_21.json')
    search(i, index_22, model_22, qb, k=100, run_path=f'{path}query_{j}/run_22.json')
    search(i, index_23, model_23, qb, k=100, run_path=f'{path}query_{j}/run_23.json')
    search(i, index_24, model_24, qb, k=100, run_path=f'{path}query_{j}/run_24.json')
    search(i, index_25, model_25, qb, k=100, run_path=f'{path}query_{j}/run_25.json')

## Preprocess all the synthetic searches and turn them into TREC style files

In [ ]:
# List all the JSON outputs of each run

path = '/Users/user/question-retrieval-KIPerWeb/runs/ANN_LM-TFIDF-Weighted-AVG/json_format/'
json_files = list_files_from_dir(path, extention='.json')
# json_files

In [ ]:
# def validate_length(list_of_files):
#     assert len(list_of_files) == 375, print(len(list_of_files))
#     for e in list_of_files:
#         assert 100 == len(read_json(e)), print(f"{e} -> didnt match the a length of 100, it returned less:", len(read_json(e)))
    
        
# validate_length(json_files)

In [ ]:
# For each JSON output, transform the results to a TREC format

for e in json_files:
    try:
        to_trec_format(e)
    except IndexError:
        print(f"Error! in {e}")

In [ ]:
#list the TREC formatted files, these have a .tsv extention
list_files_from_dir(path, extention='.tsv')

# Process all the runs and continue with the trec formatting
path_list_processed = list_files_from_dir(path, extention='.tsv')

list_of_dfs = []
for e in path_list_processed:
    df = pd.concat([pd.read_csv(e, header=None, sep='\t', names=["queries", "Q0", "docid",
                                                                 "rank", "score", "tag"])])
    list_of_dfs.append(df)
    


all_runs = pd.concat(list_of_dfs).reset_index(drop=True)


all_runs= all_runs.merge(queries_file,on='queries')
# all_runs = all_runs[["qid", "Q0", "docid", "rank", "score", "tag"]]
all_runs = all_runs[["qid", "Q0", "docid", "score", "rank", "tag"]]


all_runs['qid'] = all_runs['qid'].astype(str)
all_runs['docid'] = all_runs['docid'].astype(str)

all_runs

In [ ]:
# Extract runs by different LMs
queries = list(set(all_runs['qid']))
L_models = list(set(all_runs['tag']))
runs = [all_runs.query(f'tag=="{e}"') for e in L_models]

for i, j in zip(runs, L_models):
    i.to_csv(f'/Users/user/question-retrieval-KIPerWeb/runs/ANN_LM-TFIDF-Weighted-AVG/trec_format/{j}_run.txt', header = None, sep='\t',index=False )


In [ ]:
f = "/Users/user/question-retrieval-KIPerWeb/testbeds/queries_experiments/trec_pools/testbed.csv"

qrels_df = pd.read_csv(f)
qrels_df['qid'] = qrels_df['qid'].astype(str)
qrels_df['docid'] = qrels_df['docid'].astype(str)

qrels = Qrels.from_df(
    df=qrels_df,
    q_id_col="qid",
    doc_id_col="docid",
    score_col="qrel",
)

In [ ]:
path = '/Users/user/question-retrieval-KIPerWeb/runs/ANN_LM-TFIDF-Weighted-AVG/trec_format/'
trec_runs = list_files_from_dir(path, extention='.txt')
trec_runs

runs = []
for e in trec_runs:
    runs.append(Run.from_file(e))
    
runs_names = []
for e in trec_runs:
    runs_names.append(get_last_element_of_path(e))


In [ ]:
report = create_report(qrels, runs, runs_names, 25,"/Users/user/question-retrieval-KIPerWeb/data/results/ANN_LM-TFIDF-Weighted-AVG/ann_tfidf_avg_metrics_at_25.csv")
report.sort_values(by='ndcg@25',  ascending=False).head(5)

In [ ]:
report = create_report(qrels, runs, runs_names, 50,"/Users/user/question-retrieval-KIPerWeb/data/results/ANN_LM-TFIDF-Weighted-AVG/ann_tfidf_avg_metrics_at_50.csv")
report.sort_values(by='ndcg@50',  ascending=False).head(5)

In [ ]:
report = create_report(qrels, runs, runs_names, 100,"/Users/user/question-retrieval-KIPerWeb/data/results/ANN_LM-TFIDF-Weighted-AVG/ann_tfidf_avg_metrics_at_100.csv")
report.sort_values(by='ndcg@100',  ascending=False).head(5)